# The best model for TPSAPR22 without DL

What does this notebook do?
- We extract features using tsflex `tsflex`: https://github.com/predict-idlab/tsflex  
- We perform feature selection using `powershap`: https://github.com/predict-idlab/powershap  
- We fit 1 final catboost model on the collection of features (in this notebook)


# <a style="color:orange">The feature extraction & feature selection libraries</a>

## [**tsflex**](https://github.com/predict-idlab/tsflex)
> #### `tsflex` is a toolkit for **flex**ible **t**ime **s**eries processing & feature extraction, that is highly efficient  

`tsflex` is flexible as it: 
 * handles multivariate time series
 * **integrates** with many other libraries (numpy, scipy, seglearn, tsfresh, tsfel, catch22, ...)
 * coveniently allows to extract features on **multiple step & window sizes**

## [**powershap**](https://github.com/predict-idlab/powershap)
> #### `powershap` is a **feature selection method** that uses statistical hypothesis testing and power calculations on **Shapley values**, enabling fast and intuitive wrapper-based feature selection.  

<br>

---


Some features have been inspired by [AMBROSM's notebook](https://www.kaggle.com/code/ambrosm/tpsapr22-best-model-without-nn)

<br>

In [2]:
!pip install tsflex
!pip install seglearn antropy catch22
!pip install powershap

  Using cached multiprocess-0.70.12.2-py38-none-any.whl (128 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\0stix\\anaconda3\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.5.0 requires daal==2021.4.0, which is not installed.
catboost 0.26.1 requires graphviz, which is not installed.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.3.0 requires gast==0.3.3, but you have gast 0.4.0 which is incompatible.
tensorflow 2.3.0 requires numpy<1.19.0,>=1.16.0, but you have numpy 1.20.3 which is incompatible.
tensorflow 2.3.0 requires scipy==1.4.1, but you have scipy 1.5.4 which is incompatible.
tensorflow 2.3.0 requires tensorflow-estimator<2.4.0,>=2.3.0, but you have tensorflow-estimator 2.6.0 which is incompatible.
tensorboard 2.4.0 requires google-auth<2,>=1.6.3, but you have google-auth 2.6.0 which is incompatible.
tensorboard 2.4.0 requires google-auth-oauthlib<0.5,>=0.4.1, but you have google-auth-oauthlib 0.5

  Using cached numpy-1.20.3-cp38-cp38-win_amd64.whl (13.7 MB)
  Created wheel for antropy: filename=antropy-0.1.4-py3-none-any.whl size=16162 sha256=b2a2bebcb1478fffea05068b37616e341910e666088886b216710b8d73eecd21
  Stored in directory: c:\users\0stix\appdata\local\pip\cache\wheels\08\df\79\75844268b16208a10ed10d0bfab8814deb9698a47f715a25a5
  Created wheel for catch22: filename=catch22-0.2.0-cp38-cp38-win_amd64.whl size=36169 sha256=99991b8da8abe22c52e5720f4823fe5f63234e2524b2dfb62d6f9c8f105974c0
  Stored in directory: c:\users\0stix\appdata\local\pip\cache\wheels\75\00\35\45bd9f66714d1c9a7513ef2259ec0d559522f48d446727b6d7
Successfully built antropy catch22
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.3
    Uninstalling numpy-1.22.3:
      Successfully uninstalled numpy-1.22.3
  Using cached sklearn-0.0.tar.gz (1.1 kB)
  Using cached statsmodels-0.13.2-cp38-cp38-win_amd64.whl (9.1 MB)
  Using cached shap-0.40.0-cp38-cp38-win_amd64.whl (431 kB)
  Using cache

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\0stix\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from cycler import cycler
from IPython.display import display
from tqdm.notebook import tqdm
import datetime
import scipy.stats

from sklearn.model_selection import GroupKFold, cross_val_score, GroupShuffleSplit
from sklearn.ensemble import HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.pipeline import make_pipeline
from catboost import CatBoostClassifier

# Our feature extraction library
from tsflex.features import FeatureCollection, MultipleFeatureDescriptors, FuncWrapper
from tsflex.features.integrations import seglearn_feature_dict_wrapper, tsfresh_settings_wrapper, catch22_wrapper

# Our feature selection library
from powershap import PowerShap

# The feature function libraries
from seglearn.feature_functions import base_features
from tsfresh.feature_extraction import ComprehensiveFCParameters
import antropy as ant
from catch22 import catch22_all

In [3]:
import warnings

warnings.filterwarnings("ignore", category=RuntimeWarning)

In [4]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

## Load the data

In [5]:
# Reading the raw data
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')

# Merge the labels into the train data
train = train.merge(train_labels, how='left', on="sequence")

sensors = [col for col in train.columns if 'sensor_' in col]

train.shape, test.shape

((1558080, 17), (733080, 16))

In [6]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

train_pivoted0 = train.pivot(index=['sequence', 'subject'], columns='step', values=sensors)
display(train_pivoted0)

Memory usage of dataframe is 213.97 MB
Memory usage after optimization is: 112.93 MB
Decreased by 47.2%
Memory usage of dataframe is 89.49 MB
Memory usage after optimization is: 44.74 MB
Decreased by 50.0%


sensor_00                                                    \
step                    0         1         2         3         4         5    
sequence subject                                                               
0        47      -0.196291 -0.447450  0.326893  0.523184  0.272025 -1.073416   
1        66      -6.657650  1.633694  1.863215 -2.846213  0.593508  2.469861   
2        66      -1.741886  3.402627 -0.632148  0.208655 -0.800618 -2.274343   
3        542      0.264297 -1.808346  0.078825  1.037094 -0.189335 -1.302937   
4        437      0.567233 -0.823802  1.476816 -1.710201 -0.408810  0.556414   
...                    ...       ...       ...       ...       ...       ...   
25963    558     -1.441267  2.359351 -0.881762  1.299073 -0.862442 -1.428130   
25964    297      0.996909  0.415765  1.874034 -5.220247  2.927357 -0.311437   
25965    129     -0.097372 -0.860896  0.928903  0.750386  1.346986 -3.422720   
25966    170     -0.357032 -1.603555 -0.296754  0.273570  0.462133  0.244977   
25967    327     -0.313756  0.392581 -0.554096  0.070325 -0.258114  0.095054   

                                                          ...  sensor_12  \
step                    6         7         8         9   ...         50   
sequence subject                                          ...              
0        47       0.350850 -0.304482 -0.216383  0.303709  ...  -1.670077   
1        66      -1.202473 -0.829212  1.091190 -0.334621  ...  -0.338022   
2        66       2.009274 -0.563369 -0.050232  0.800618  ...  -0.172634   
3        542      1.078825  2.257342 -2.283617 -2.759660  ...   0.145354   
4        437      0.853168 -0.406491 -0.009274 -0.021638  ...   0.001705   
...                    ...       ...       ...       ...  ...        ...   
25963    558      0.125193  2.119784  0.208655  1.847759  ...   0.049020   
25964    297      0.496909  0.372488 -0.341577 -0.565688  ...  36.889172   
25965    129      0.983771  2.818393 -1.460587 -1.988408  ...   2.394714   
25966    170     -0.341577 -0.229521  0.877125 -1.197063  ...  14.010656   
25967    327      0.387172 -0.824575  0.471406 -0.037094  ...   0.909207   

                                                                         \
step                     51         52         53         54         55   
sequence subject                                                          
0        47       -5.306479   3.621057   4.965047  -3.615089   0.791134   
1        66        0.234868   0.030691   0.037511  -0.120205   0.056692   
2        66        0.091645  -0.028986  -0.202472   0.332481  -0.013214   
3        542      -0.045610  -0.098039  -0.049446   0.055840  -0.022165   
4        437      -0.050725  -0.025149   0.106138  -0.098465  -0.040068   
...                     ...        ...        ...        ...        ...   
25963    558      -0.089940   0.059250  -0.013640  -0.083973   0.026002   
25964    297      21.105286  -4.421143  16.871696  -4.326940 -13.625319   
25965    129      -0.677323   1.435209  -0.645780  -0.449275   0.271952   
25966    170      48.826939  25.958227  -7.077153 -12.375533   9.170076   
25967    327      -0.830776  -1.007246   1.064365   0.639812  -0.914748   

                                                             
step                    56         57         58         59  
sequence subject                                             
0        47       5.887042  -7.211424 -26.738705 -50.658993  
1        66       0.062234  -0.013640   0.098039  -0.115090  
2        66       0.245524  -0.091219  -0.306905   0.046036  
3        542      0.054135  -0.185422   0.311168  -0.166240  
4        437      0.135976   0.121910  -0.059676  -0.087809  
...                    ...        ...        ...        ...  
25963    558      0.036232  -0.042199   0.028986  -0.015772  
25964    297      9.957375 -13.389174 -17.123615   9.219522  
25965    129     -0.262575   0.713129  -1.378517   1.802217  
25966    170      6.789855 -15.28729

# Feature extraction with **tsflex**

## Extract the same features as in [this notebook](https://www.kaggle.com/code/ambrosm/tpsapr22-best-model-without-nn)

Let's keep it simple and calculate only the following features:
- For every sensor, we calculate mean, standard deviation, interquartile range, standard deviation divided by mean, and kurtosis. This gives the first 5\*13=65 features.
- For the special sensor_02, we count how many times it goes up or down.
- For sensor_02, we calculate the sum of all upward / downward steps, the maximum of all upward / downward steps, and the mean of all upward / downward steps. 
- For every subject, we count how many sequences belong to it, and we add this count as a feature to all sequences of the subject (the [EDA](https://www.kaggle.com/code/ambrosm/tpsapr22-eda-which-makes-sense) gives the motivation for this feature). 

Now we have 74 features. 

In [7]:
# Feature engineering
def engineer(df):
    new_df = pd.DataFrame([], index=df.index)
    for sensor in sensors:
        new_df[sensor + '_mean'] = df[sensor].mean(axis=1)
        new_df[sensor + '_std'] = df[sensor].std(axis=1)
        new_df[sensor + '_iqr'] = scipy.stats.iqr(df[sensor], axis=1)
        new_df[sensor + '_sm'] = np.nan_to_num(new_df[sensor + '_std'] / 
                                               new_df[sensor + '_mean'].abs()).clip(-1e30, 1e30)
        new_df[sensor + '_kurtosis'] = scipy.stats.kurtosis(df[sensor], axis=1)
    new_df['sensor_02_up'] = (df.sensor_02.diff(axis=1) > 0).sum(axis=1)
    new_df['sensor_02_down'] = (df.sensor_02.diff(axis=1) < 0).sum(axis=1)
    new_df['sensor_02_upsum'] = df.sensor_02.diff(axis=1).clip(0, None).sum(axis=1)
    new_df['sensor_02_downsum'] = df.sensor_02.diff(axis=1) .clip(None, 0).sum(axis=1)
    new_df['sensor_02_upmax'] = df.sensor_02.diff(axis=1).max(axis=1)
    new_df['sensor_02_downmax'] = df.sensor_02.diff(axis=1).min(axis=1)
    new_df['sensor_02_upmean'] = np.nan_to_num(new_df['sensor_02_upsum'] / new_df['sensor_02_up'], posinf=40)
    new_df['sensor_02_downmean'] = np.nan_to_num(new_df['sensor_02_downsum'] / new_df['sensor_02_down'], neginf=-40)
    return new_df

train_pivoted = engineer(train_pivoted0)

train_shuffled = train_pivoted.sample(frac=1.0, random_state=1)
labels_shuffled = train_labels.reindex(train_shuffled.index.get_level_values('sequence'))
labels_shuffled = labels_shuffled[['state']].merge(train[['sequence', 'subject']].groupby('sequence').min(),
                                                   how='left', on='sequence')
labels_shuffled = labels_shuffled.merge(labels_shuffled.groupby('subject').size().rename('sequence_count'),
                                        how='left', on='subject')
train_shuffled['sequence_count_of_subject'] = labels_shuffled['sequence_count'].values

selected_columns = train_shuffled.columns
print(len(selected_columns))
#train_shuffled.columns

74


## Extract base features from seglearn

In [8]:
basic_feats = MultipleFeatureDescriptors(
        functions=seglearn_feature_dict_wrapper(base_features()),
        series_names=sensors,
        windows=60,
        strides=60,
    )

fc_seglearn = FeatureCollection(basic_feats)

df_feats = fc_seglearn.calculate(train, show_progress=True, return_df=True, window_idx="begin")
df_feats.index = train['sequence'].unique()

  0%|          | 0/143 [00:00<?, ?it/s]

In [9]:
train_shuffled = train_shuffled.reset_index().merge(df_feats, left_on='sequence', right_index=True)
train_shuffled = train_shuffled.set_index(['sequence', 'subject'])

## Load extracted & selected tsfresh features

The selection of these features were performed locally using `powershap`

In [10]:
# create the tsfresh feature extractor
settings = ComprehensiveFCParameters()  # all the tsfresh features
del settings["linear_trend_timewise"]  # requires a time-index

fc_tsfresh = FeatureCollection(
    MultipleFeatureDescriptors(
        functions=tsfresh_settings_wrapper(settings),
        series_names=sensors,
        windows=60,
        strides=60
    )
)

In [11]:
tsfresh_selected_cols = ["sensor_00__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_00__ar_coefficient_{'coeff': 3, 'k': 10}__w=60_s=60",
       "sensor_00__ar_coefficient_{'coeff': 4, 'k': 10}__w=60_s=60",
       "sensor_00__augmented_dickey_fuller_{'attr': 'usedlag'}__w=60_s=60",
       "sensor_01__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_01__ar_coefficient_{'coeff': 1, 'k': 10}__w=60_s=60",
       "sensor_01__ar_coefficient_{'coeff': 2, 'k': 10}__w=60_s=60",
       "sensor_01__ar_coefficient_{'coeff': 3, 'k': 10}__w=60_s=60",
       "sensor_01__ar_coefficient_{'coeff': 4, 'k': 10}__w=60_s=60",
       "sensor_01__ar_coefficient_{'coeff': 6, 'k': 10}__w=60_s=60",
       "sensor_01__cwt_coefficients_{'widths': (2, 5, 10, 20), 'coeff': 10, 'w': 10}__w=60_s=60",
       "sensor_01__fft_coefficient_{'coeff': 1, 'attr': 'imag'}__w=60_s=60",
       "sensor_01__spkt_welch_density_{'coeff': 2}__w=60_s=60",
       "sensor_02__agg_linear_trend_{'attr': 'intercept', 'chunk_len': 10, 'f_agg': 'var'}__w=60_s=60",
       "sensor_02__agg_linear_trend_{'attr': 'slope', 'chunk_len': 10, 'f_agg': 'var'}__w=60_s=60",
       "sensor_02__agg_linear_trend_{'attr': 'stderr', 'chunk_len': 5, 'f_agg': 'var'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.0, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.0, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.2, 'qh': 0.6, 'isabs': False, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.2, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.6, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.8, 'qh': 1.0, 'isabs': False, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_02__change_quantiles_{'ql': 0.8, 'qh': 1.0, 'isabs': True, 'f_agg': 'var'}__w=60_s=60",
       "sensor_02__cid_ce_{'normalize': True}__w=60_s=60",
       'sensor_02__mean_abs_change__w=60_s=60',
       "sensor_02__number_peaks_{'n': 1}__w=60_s=60",
       "sensor_02__partial_autocorrelation_{'lag': 2}__w=60_s=60",
       "sensor_03__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_03__ar_coefficient_{'coeff': 5, 'k': 10}__w=60_s=60",
       "sensor_04__agg_autocorrelation_{'f_agg': 'var', 'maxlag': 40}__w=60_s=60",
       "sensor_04__agg_linear_trend_{'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'min'}__w=60_s=60",
       "sensor_04__agg_linear_trend_{'attr': 'rvalue', 'chunk_len': 10, 'f_agg': 'var'}__w=60_s=60",
       "sensor_04__agg_linear_trend_{'attr': 'rvalue', 'chunk_len': 5, 'f_agg': 'min'}__w=60_s=60",
       "sensor_04__approximate_entropy_{'m': 2, 'r': 0.5}__w=60_s=60",
       "sensor_04__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_04__ar_coefficient_{'coeff': 10, 'k': 10}__w=60_s=60",
       "sensor_04__ar_coefficient_{'coeff': 2, 'k': 10}__w=60_s=60",
       "sensor_04__ar_coefficient_{'coeff': 4, 'k': 10}__w=60_s=60",
       "sensor_04__augmented_dickey_fuller_{'attr': 'usedlag'}__w=60_s=60",
       "sensor_04__autocorrelation_{'lag': 7}__w=60_s=60",
       "sensor_04__cid_ce_{'normalize': True}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 0}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 2}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 3}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 4}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 5}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 6}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 7}__w=60_s=60",
       "sensor_04__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 9}__w=60_s=60",
       "sensor_04__fft_aggregated_{'aggtype': 'kurtosis'}__w=60_s=60",
       "sensor_04__fft_aggregated_{'aggtype': 'skew'}__w=60_s=60",
       "sensor_04__fft_coefficient_{'coeff': 3, 'attr': 'abs'}__w=60_s=60",
       "sensor_04__fourier_entropy_{'bins': 100}__w=60_s=60",
       "sensor_04__friedrich_coefficients_{'coeff': 1, 'm': 3, 'r': 30}__w=60_s=60",
       "sensor_04__friedrich_coefficients_{'coeff': 3, 'm': 3, 'r': 30}__w=60_s=60",
       "sensor_04__index_mass_quantile_{'q': 0.2}__w=60_s=60",
       'sensor_04__kurtosis__w=60_s=60',
       "sensor_04__large_standard_deviation_{'r': 0.25}__w=60_s=60",
       'sensor_04__longest_strike_above_mean__w=60_s=60',
       "sensor_04__number_peaks_{'n': 10}__w=60_s=60",
       "sensor_04__number_peaks_{'n': 5}__w=60_s=60",
       "sensor_04__ratio_beyond_r_sigma_{'r': 0.5}__w=60_s=60",
       "sensor_04__ratio_beyond_r_sigma_{'r': 1}__w=60_s=60",
       "sensor_04__ratio_beyond_r_sigma_{'r': 2}__w=60_s=60",
       "sensor_04__spkt_welch_density_{'coeff': 2}__w=60_s=60",
       "sensor_04__time_reversal_asymmetry_statistic_{'lag': 2}__w=60_s=60",
       "sensor_05__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_05__ar_coefficient_{'coeff': 2, 'k': 10}__w=60_s=60",
       "sensor_05__fft_coefficient_{'coeff': 0, 'attr': 'abs'}__w=60_s=60",
       "sensor_05__fft_coefficient_{'coeff': 4, 'attr': 'abs'}__w=60_s=60",
       "sensor_06__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_06__fft_coefficient_{'coeff': 1, 'attr': 'imag'}__w=60_s=60",
       "sensor_06__spkt_welch_density_{'coeff': 2}__w=60_s=60",
       "sensor_07__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_07__ar_coefficient_{'coeff': 5, 'k': 10}__w=60_s=60",
       'sensor_07__skewness__w=60_s=60',
       "sensor_07__spkt_welch_density_{'coeff': 2}__w=60_s=60",
       "sensor_09__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_09__ar_coefficient_{'coeff': 5, 'k': 10}__w=60_s=60",
       "sensor_09__augmented_dickey_fuller_{'attr': 'usedlag'}__w=60_s=60",
       "sensor_09__fft_coefficient_{'coeff': 1, 'attr': 'imag'}__w=60_s=60",
       "sensor_09__spkt_welch_density_{'coeff': 2}__w=60_s=60",
       "sensor_10__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_10__ar_coefficient_{'coeff': 10, 'k': 10}__w=60_s=60",
       "sensor_10__augmented_dickey_fuller_{'attr': 'usedlag'}__w=60_s=60",
       "sensor_10__autocorrelation_{'lag': 2}__w=60_s=60",
       "sensor_10__autocorrelation_{'lag': 5}__w=60_s=60",
       "sensor_10__cid_ce_{'normalize': True}__w=60_s=60",
       "sensor_10__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 1}__w=60_s=60",
       "sensor_10__energy_ratio_by_chunks_{'num_segments': 10, 'segment_focus': 3}__w=60_s=60",
       "sensor_10__fft_coefficient_{'coeff': 3, 'attr': 'abs'}__w=60_s=60",
       "sensor_10__fft_coefficient_{'coeff': 4, 'attr': 'abs'}__w=60_s=60",
       "sensor_10__fft_coefficient_{'coeff': 6, 'attr': 'abs'}__w=60_s=60",
       "sensor_10__fourier_entropy_{'bins': 100}__w=60_s=60",
       'sensor_10__kurtosis__w=60_s=60',
       "sensor_10__linear_trend_{'attr': 'pvalue'}__w=60_s=60",
       "sensor_10__partial_autocorrelation_{'lag': 3}__w=60_s=60",
       "sensor_10__partial_autocorrelation_{'lag': 4}__w=60_s=60",
       "sensor_10__spkt_welch_density_{'coeff': 2}__w=60_s=60",
       "sensor_11__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_11__ar_coefficient_{'coeff': 1, 'k': 10}__w=60_s=60",
       "sensor_12__ar_coefficient_{'coeff': 0, 'k': 10}__w=60_s=60",
       "sensor_12__ar_coefficient_{'coeff': 1, 'k': 10}__w=60_s=60",
       "sensor_12__ar_coefficient_{'coeff': 10, 'k': 10}__w=60_s=60",
       "sensor_12__ar_coefficient_{'coeff': 2, 'k': 10}__w=60_s=60",
       "sensor_12__augmented_dickey_fuller_{'attr': 'usedlag'}__w=60_s=60",
       "sensor_12__autocorrelation_{'lag': 2}__w=60_s=60",
       "sensor_12__change_quantiles_{'ql': 0.4, 'qh': 0.6, 'isabs': True, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_12__change_quantiles_{'ql': 0.8, 'qh': 1.0, 'isabs': True, 'f_agg': 'mean'}__w=60_s=60",
       "sensor_12__fft_aggregated_{'aggtype': 'skew'}__w=60_s=60",
       "sensor_12__fft_coefficient_{'coeff': 0, 'attr': 'abs'}__w=60_s=60",
       "sensor_12__fourier_entropy_{'bins': 100}__w=60_s=60",
       "sensor_12__partial_autocorrelation_{'lag': 3}__w=60_s=60",
       "sensor_12__ratio_beyond_r_sigma_{'r': 2}__w=60_s=60"]


df_feats_tsfresh = pd.read_parquet('../input/tsflex-x-tsfresh-feature-extraction/tsfresh_feats_train.parquet')[tsfresh_selected_cols]
df_feats_tsfresh.index = train['sequence'].unique()

In [12]:
already_included = list(set(train_shuffled.columns).intersection(df_feats_tsfresh.columns))
print(already_included)
df_feats_tsfresh = df_feats_tsfresh.drop(columns=already_included)

[]


In [13]:
train_shuffled = train_shuffled.reset_index().merge(df_feats_tsfresh, left_on='sequence', right_index=True)
train_shuffled = train_shuffled.set_index(['sequence', 'subject'])

## Extract antropy features

In [14]:
time_funcs = [
    ant.svd_entropy, ant.perm_entropy, ant.katz_fd, ant.higuchi_fd, ant.petrosian_fd
]

fc_antropy = FeatureCollection(
    MultipleFeatureDescriptors(
        functions=time_funcs,
        series_names=sensors,
        windows=60,
        strides=60
    )
)

df_feats_antropy = fc_antropy.calculate(train.astype(np.float32), return_df=True, show_progress=True)
df_feats_antropy.index = train['sequence'].unique()

  0%|          | 0/65 [00:00<?, ?it/s]

In [15]:
already_included = list(set(train_shuffled.columns).intersection(df_feats_antropy.columns))
print(already_included)
df_feats_antropy = df_feats_antropy.drop(columns=already_included)

[]


In [16]:
train_shuffled = train_shuffled.reset_index().merge(df_feats_antropy, left_on='sequence', right_index=True)
train_shuffled = train_shuffled.set_index(['sequence', 'subject'])

## Extract catch22 features

In [17]:
fc_catch22 = FeatureCollection(
    MultipleFeatureDescriptors(
        functions=catch22_wrapper(catch22_all),
        series_names=sensors,
        windows=60,
        strides=60
    )
)

df_feats_train_catch22 = fc_catch22.calculate(train, show_progress=True, return_df=True, window_idx="begin")
df_feats_train_catch22.index =  train["sequence"].unique()

  0%|          | 0/13 [00:00<?, ?it/s]

In [18]:
already_included = list(set(train_shuffled.columns).intersection(df_feats_train_catch22.columns))
print(already_included)
df_feats_train_catch22 = df_feats_train_catch22.drop(columns=already_included)

[]


In [19]:
train_shuffled = train_shuffled.reset_index().merge(df_feats_train_catch22, left_on='sequence', right_index=True)
train_shuffled = train_shuffled.set_index(['sequence', 'subject'])

# Feature selection with **powershap**

In [20]:
# Drop some useless features: see https://www.kaggle.com/code/ambrosm/tpsapr22-best-model-without-nn
dropped_features = ['sensor_05_kurtosis', 'sensor_08_mean',
                    'sensor_05_std', 'sensor_06_kurtosis',
                    'sensor_06_std', 'sensor_03_std',
                    'sensor_02_kurtosis', 'sensor_03_kurtosis',
                    'sensor_09_kurtosis', 'sensor_03_mean',
                    'sensor_00_mean', 'sensor_02_iqr',
                    'sensor_05_mean', 'sensor_06_mean',
                    'sensor_07_std', 'sensor_10_iqr',
                    'sensor_11_iqr', 'sensor_12_iqr',
                    'sensor_09_mean', 'sensor_02_sm',
                    'sensor_03_sm', 'sensor_05_iqr', 
                    'sensor_06_sm', 'sensor_09_iqr', 
                    'sensor_07_iqr', 'sensor_10_mean']
selected_columns = [f for f in selected_columns if f not in dropped_features]
len(selected_columns)

48

In [21]:
seglearn_features = list(df_feats.columns)
selected_columns += seglearn_features

The feature selection for the tsfresh features has been performed by [powershap](https://github.com/predict-idlab/powershap)

In [22]:
tsfresh_features = list(df_feats_tsfresh.columns)
selected_columns += tsfresh_features

In [23]:
antropy_features = list(df_feats_antropy.columns)
selected_columns += antropy_features

In [24]:
catch22_feature_names = list(df_feats_train_catch22.columns)
if 'sequence' in catch22_feature_names:
    catch22_feature_names.remove('sequence')
    
selected_columns += catch22_feature_names

In [25]:
len(selected_columns)

656

#### Now apply **powershap** for feature selection

In [26]:
selector = PowerShap()
selector.fit(train_shuffled[selected_columns], labels_shuffled["state"])


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

PowerShap(model=<catboost.core.CatBoostClassifier object at 0x7fbfd3eda310>)

In [27]:
selected_columns_p = np.array(selected_columns)[selector._get_support_mask()]
len(selected_columns_p)

154

### Extract the selected features on the test set

In [28]:
fc_all = FeatureCollection(
    [
        fc_seglearn,
        fc_tsfresh.reduce(tsfresh_selected_cols),
        fc_antropy,
        fc_catch22,
    ]
)

selected_columns_p_tsflex = [c for c in selected_columns_p if c.endswith("_w=60_s=60")]
print(len(selected_columns_p_tsflex))
fc_selected = fc_all.reduce(selected_columns_p_tsflex)

138


In [29]:
df_feats_test = fc_selected.calculate(test, return_df=True, show_progress=True)
print(df_feats_test.shape)  # Some features return multiple outputs (that is why this shape is way larger than # selected cols)
df_feats_test = df_feats_test[selected_columns_p_tsflex]
df_feats_test.index = test["sequence"].unique()
df_feats_test.shape

  0%|          | 0/88 [00:00<?, ?it/s]

(12218, 2591)


(12218, 138)

In [30]:
# Feature engineering for test

test_pivoted0 = test.pivot(index=['sequence', 'subject'], columns='step', values=sensors)
# Manual feature extraction (cfr. AmbrosM)
test_pivoted = engineer(test_pivoted0)

# Add the selected features from the reduced tsflex feature collection
test_pivoted = test_pivoted.reset_index().merge(df_feats_test, left_on='sequence', right_index=True)
test_pivoted = test_pivoted.set_index(['sequence', 'subject'])

sequence_count = test_pivoted.index.to_frame(index=False).groupby('subject').size().rename('sequence_count_of_subject')
submission = pd.DataFrame({'sequence': test_pivoted.index.get_level_values('sequence')})
test_pivoted = test_pivoted.reset_index()
test_pivoted = test_pivoted.merge(sequence_count, how='left', on='subject')
test_pivoted.head(2)

,sequence,subject,sensor_00_mean,sensor_00_std,sensor_00_iqr,sensor_00_sm,sensor_00_kurtosis,sensor_01_mean,sensor_01_std,sensor_01_iqr,...,sensor_07__SP_Summaries_welch_rect_area_5_1__w=60_s=60,sensor_10__CO_Embed2_Dist_tau_d_expfit_meandiff__w=60_s=60,sensor_10__CO_FirstMin_ac__w=60_s=60,sensor_10__SP_Summaries_welch_rect_area_5_1__w=60_s=60,sensor_11__DN_OutlierInclude_n_001_mdrmd__w=60_s=60,sensor_11__SP_Summaries_welch_rect_area_5_1__w=60_s=60,sensor_12__CO_Embed2_Dist_tau_d_expfit_meandiff__w=60_s=60,sensor_12__SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1__w=60_s=60,sensor_12__SP_Summaries_welch_rect_area_5_1__w=60_s=60,sequence_count_of_subject
0,25968,684,-0.002602,1.091741,1.078632,419.616577,6.400713,-0.036432,4.499484,0.990284,...,0.023508,0.074308,2.0,0.074504,-0.250000,0.032295,0.052997,0.730769,0.003189,70
1,25969,935,0.028516,1.450040,0.844281,50.849651,6.634404,0.002276,0.775379,1.079832,...,0.005527,0.104384,2.0,0.025742,0.033333,0.016460,0.141736,0.769231,0.013637,59


# Cross validation

In [31]:
assert all([c in test_pivoted.columns.values for c in selected_columns_p])

In [32]:
%%time

# Cross-validation of the classifier

print(f"{len(selected_columns_p)} features")
score_list = []
kf = GroupKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(train_shuffled, groups=train_shuffled.index.get_level_values('subject'))):
    X_tr = train_shuffled.iloc[idx_tr][selected_columns_p]
    X_va = train_shuffled.iloc[idx_va][selected_columns_p]
    y_tr = labels_shuffled.iloc[idx_tr].state
    y_va = labels_shuffled.iloc[idx_va].state
    
    train_groups = train_shuffled.index.get_level_values('subject')[idx_tr]
    
    kf2 = GroupKFold(n_splits=10)
    idx_cv_tr, idx_cv_val = next(kf.split(X_tr, groups=train_groups))
    X_cv_tr = X_tr.iloc[idx_cv_tr]
    X_cv_va = X_tr.iloc[idx_cv_val]
    y_cv_tr = y_tr.iloc[idx_cv_tr]
    y_cv_val = y_tr.iloc[idx_cv_val]

    model = CatBoostClassifier(iterations=5000, verbose=0, od_type='Iter', od_wait=100, task_type="CPU")
    model.fit(X_cv_tr, y_cv_tr, eval_set=(X_cv_va, y_cv_val))
    
    best_iter = model.get_best_iteration()
    
    model = CatBoostClassifier(iterations=best_iter+100, verbose=0, 
                               learning_rate=model._learning_rate,
                               od_type='Iter', od_wait=100, task_type="CPU")
    model.fit(X_tr, y_tr)

    y_va_pred = model.predict_proba(X_va.values)[:,1]
    score = roc_auc_score(y_va, y_va_pred)
    try:
        print(f"Fold {fold}: n_iter ={best_iter:5d}    AUC = {score:.4f}")
    except AttributeError:
        print(f"Fold {fold}:                  AUC = {score:.3f}")
    score_list.append(score)
    
print(f"OOF AUC:                       {np.mean(score_list):.4f}")


154 features
Fold 0: n_iter = 1499    AUC = 0.9740
Fold 1: n_iter = 1487    AUC = 0.9710
Fold 2: n_iter = 1899    AUC = 0.9757
Fold 3: n_iter = 1381    AUC = 0.9754
Fold 4: n_iter = 1973    AUC = 0.9695
OOF AUC:                       0.9731
CPU times: user 27min 20s, sys: 54.9 s, total: 28min 15s
Wall time: 7min 23s


## Generate submission

In [33]:
# Retrain, predict and write submission
print(f"{len(selected_columns_p)} features")

pred_list = []
for seed in range(10):
    X_tr = train_shuffled[selected_columns_p]
    y_tr = labels_shuffled.state

    kf2 = GroupShuffleSplit(n_splits=10, random_state=seed)
    idx_cv_tr, idx_cv_val = next(kf2.split(X_tr, groups=train_shuffled.index.get_level_values('subject')))
    X_cv_tr = X_tr.iloc[idx_cv_tr]
    X_cv_va = X_tr.iloc[idx_cv_val]
    y_cv_tr = y_tr.iloc[idx_cv_tr]
    y_cv_val = y_tr.iloc[idx_cv_val]
    
    model = CatBoostClassifier(iterations=5000, verbose=0, od_type='Iter', 
                               random_state=seed,
                               od_wait=100, task_type="CPU")
    model.fit(X_cv_tr, y_cv_tr, eval_set=(X_cv_va, y_cv_val))
    
    best_iter = model.get_best_iteration()
    
    model = CatBoostClassifier(iterations=best_iter+100, verbose=0, 
                               learning_rate=model._learning_rate,
                               random_state=seed,
                               od_type='Iter', od_wait=100, task_type="CPU")
    model.fit(X_tr, y_tr)
    pred_list.append(scipy.stats.rankdata(model.predict_proba(test_pivoted[selected_columns_p].values)[:, 1]))
    
    print(f"{seed:2}", pred_list[-1])
    print()
submission['state'] = sum(pred_list) / len(pred_list)
submission.to_csv('submission.csv', index=False)
submission

154 features
 0 [ 7422. 10259.  1884. ...  5527.  2217.  2290.]

 1 [ 7643. 10895.  1711. ...  5848.  2206.  2508.]

 2 [ 7797. 10621.  1744. ...  6012.  2313.  2571.]

 3 [ 8022. 10589.  2019. ...  5668.  2378.  2414.]

 4 [ 7666. 10739.  1952. ...  5692.  2346.  2463.]

 5 [ 6948. 10544.  1733. ...  5543.  2216.  2468.]

 6 [ 8128. 10234.  2114. ...  5583.  2621.  2227.]

 7 [ 7121. 10209.  1642. ...  5775.  2530.  2838.]

 8 [ 7552. 10203.  1603. ...  5401.  2168.  2854.]

 9 [ 7210. 10527.  1734. ...  5428.  2126.  2287.]



,sequence,state
0,25968,7550.9
1,25969,10482.0
2,25970,1813.6
3,25971,6585.2
4,25972,5651.2
...,...,...
12213,38181,8835.9
12214,38182,6049.9
12215,38183,5647.7
12216,38184,2312.1
